In [1]:
from dotenv import dotenv_values
from requests import get
from requests.models import HTTPError
from bs4 import BeautifulSoup

import pandas as pd

In [2]:
CONFIG = dotenv_values('../.env')

In [3]:
def requisicao_api_alura():
    try:
        response = get('https://cursos.alura.com.br/api/categorias')
        response.raise_for_status()
    except HTTPError as http_err:
        print(f'Um erro HTTP ocorreu: {http_err}')
    except Exception as err:
        print(f'Um erro ocorreu: {err}')
    else:
        return response.json()

In [4]:
def subcategorias():
    dados = requisicao_api_alura()
    url = 'https://cursos.alura.com.br/forum/subcategoria-'
    urls_subcategorias = []
    nomes_categorias = []
    nomes_subcategorias = []
    for item in dados:
        for item_subcategoria in item['subcategorias']:
            nomes_subcategorias.append(item_subcategoria['nome'])
            nomes_categorias.append(item['nome'])
            urls_subcategorias.append(f'{url}{item_subcategoria["slug"]}/sem-resposta')
    df = pd.DataFrame({'Subcategoria': nomes_subcategorias, 'Categoria': nomes_categorias, 'Url': urls_subcategorias})
    return df

In [5]:
def requisicao(url):
    cookies = {'caelum.login.token':CONFIG['COOKIE']}
    try:
        response = get(url, cookies=cookies)
        response.raise_for_status()
    except HTTPError as http_err:
        print(f'Um erro HTTP ocorreu: {http_err}')
    except Exception as err:
        print(f'Um erro ocorreu: {err}')
    else:
        soup = BeautifulSoup(response.text,'html.parser')
        return soup

In [6]:
def roda_scraping_subcategorias(df):
    topicos_sem_resposta = []
    for url in df['Url']:
        soup = requisicao(url)
        paginacao = soup.find('nav', {'class': 'busca-paginacao-links'})
        qtd_topicos = 0
        if paginacao:
            ultima_pagina = paginacao.get_text().split()[-1]
            qtd_topicos += (int(ultima_pagina)-1)*20
            url += f'/{ultima_pagina}'
            soup = requisicao(url)
        qtd_topicos += len(soup.find_all('div', {'class': {'forumList-item-subject'}}))
        topicos_sem_resposta.append(qtd_topicos)
    df['Tópicos sem resposta'] = topicos_sem_resposta
    return df

In [24]:
def roda_scraping_off_topic():
    url = 'https://cursos.alura.com.br/forum/categoria-offtopic/sem-resposta/'
    soup = requisicao(url)
    ultima_pagina = soup.find_all('a', {'class': 'paginationLink'})[-1].text
    url = url + ultima_pagina
    soup = requisicao(url)
    qtd_topicos = (int(ultima_pagina)-1)*20 + len(soup.find_all('a', {'class': 'forumList-item-subject-info-title-link'}))
    return qtd_topicos

In [48]:
def roda_scraping_pagina_inicial():
    url = 'https://cursos.alura.com.br/forum/todos/1'
    soup = requisicao(url)
    nomes = soup.find_all('a', {'class': 'dashboard-category-name-text'})
    quantidades = soup.find_all('span', {'class': 'category-stats-item-number dashboard-stats-noReply-number'})
    
    dados = {}
    for indice in range(len(nomes)):
        dados[nomes[indice].text.strip()] = int(quantidades[indice].text.strip())
    qtd_topicos_off_topic = roda_scraping_off_topic()
    dados['Off-Topic'] = qtd_topicos_off_topic
    df = pd.DataFrame(dados.items(), columns = ['Categoria', 'Tópicos sem resposta'])
    return df

In [94]:
def adiciona_topicos_sem_subcategoria(df_subcategorias, df_pagina_inicial):
    df_pagina_inicial.drop(7, axis = 0, inplace = True)
    df_pagina_inicial.set_index('Categoria', inplace = True)
    
    df_categorias = pd.DataFrame(df_subcategorias.groupby('Categoria').sum()['Tópicos sem resposta'])
    
    df_sem_subcategoria = df_pagina_inicial - df_categorias
    sem_subcategoria = []
    for indice in df_sem_subcategoria.index:
        sem_subcategoria.append(f'{indice} - Sem subcategoria')
    df_sem_subcategoria['Subcategoria'] = sem_subcategoria
    
    df_subcategorias = df_subcategorias.append(df_sem_subcategoria.reset_index()).reset_index(drop = True)
    return df_subcategorias

In [95]:
df_pagina_inicial = roda_scraping_pagina_inicial()
df = subcategorias()
df_subcategorias = roda_scraping_subcategorias(df)

adiciona_topicos_sem_subcategoria(df_subcategorias, df_pagina_inicial)

,Subcategoria,Categoria,Url,Tópicos sem resposta
0,Lógica de programação,Programação,https://cursos.alura.com.br/forum/subcategoria...,31
1,Python,Programação,https://cursos.alura.com.br/forum/subcategoria...,59
2,Dart,Programação,https://cursos.alura.com.br/forum/subcategoria...,2
3,JavaScript,Programação,https://cursos.alura.com.br/forum/subcategoria...,2
4,Node.JS,Programação,https://cursos.alura.com.br/forum/subcategoria...,77
...,...,...,...,...
117,Front-end - Sem subcategoria,Front-end,NaN,3
118,Inovação & Gestão - Sem subcategoria,Inovação & Gestão,NaN,12
119,Mobile - Sem subcategoria,Mobile,NaN,1
120,Programação - Sem subcategoria,Programação,NaN,296
